# MODELO

## 0. CARGA DE ARCHIVOS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# Cargar en Parquet
icfesx_original = pd.read_parquet("C:/Users/CACTU/OneDrive/Escritorio/Proyectos de Análisis de Datos/Python/Icfes 2019/data/procesada/icfesx_original.parquet")
X_test = pd.read_parquet("C:/Users/CACTU/OneDrive/Escritorio/Proyectos de Análisis de Datos/Python/Icfes 2019/data/procesada/X_test.parquet")

# Recuperamos las series con el nombre correcto
y_train = pd.read_parquet("C:/Users/CACTU/OneDrive/Escritorio/Proyectos de Análisis de Datos/Python/Icfes 2019/data/procesada/y_train.parquet")["puntaje_global"]
y_test = pd.read_parquet("C:/Users/CACTU/OneDrive/Escritorio/Proyectos de Análisis de Datos/Python/Icfes 2019/data/procesada/y_test.parquet")["puntaje_global"]

print("✅ Splits cargados en Parquet")

✅ Splits cargados en Parquet


In [3]:
icfesx_original.sample(5)

,colombiano,etnia,edad,presento_fuera_edad,num_cuartos_casa,estrato_casa,internet,tv,computador,lavadora,microndas,carro,moto,consola,colegio_publico,colegio_area,nse_estudiante,nse_colegio,region_amazonica,region_andina,region_caribe,region_orinoquia,region_pacifica,internet_moderado,internet_mucho,internet_ninguno,internet_poco,internet_promedio,hogar_grande,hogar_tradicional,hogar_multiple,libros_0_10,libros_11_25,libros_26_100,libros_mas_100,lectura_0_30min,lectura_1_2h,lectura_30_60min,lectura_mas_2h,lectura_nula,edu_padre_bachiller,edu_padre_bachiller_inc,edu_padre_ninguna,edu_padre_no_aplica,edu_padre_postgrado,edu_padre_primaria,edu_padre_primaria_inc,edu_padre_profesional,edu_padre_profesional_inc,edu_padre_tecnico,edu_padre_tecnico_inc,edu_madre_bachiller,edu_madre_bachiller_inc,edu_madre_ninguna,edu_madre_no_aplica,edu_madre_postgrado,edu_madre_primaria,edu_madre_primaria_inc,edu_madre_profesional,edu_madre_profesional_inc,edu_madre_tecnico,edu_madre_tecnico_inc,act_padre_directivos,act_padre_microempresario,act_padre_no_aplica,act_padre_pensionado,act_padre_profesionales,act_padre_sector_primario,act_padre_sin_trabajo,act_padre_sin_info,act_padre_independiente,act_padre_operativos,act_madre_directivos,act_madre_microempresario,act_madre_no_aplica,act_madre_pensionado,act_madre_profesionales,act_madre_sector_primario,act_madre_sin_trabajo,act_madre_sin_info,act_madre_independiente,act_madre_operativos,trabajo_medio_tiempo,trabajo_no,trabajo_tiempo_completo,trabajo_parcial_reducido,trabajo_ocacional,colegio_femenino,colegio_masculino,colegio_mixto,jornada_completa,jornada_tradicional,jornada_unica,jornada_validacion,leche_aceptable,leche_insuficiente,leche_muy_insuficiente,leche_optimo,carne_aceptable,carne_insuficiente,carne_muy_insuficiente,carne_optimo,cereales_aceptable,cereales_insuficiente,cereales_muy_insuficiente,cereales_optimo
219242,1,0,19,0,1.00,2.00,0,0,1,0,0,0,0,0,1,1,1.00,2.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00
120586,1,0,17,0,2.00,1.00,0,0,0,1,0,0,0,0,0,1,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
281105,1,0,19,0,2.00,2.00,1,1,1,1,1,0,0,0,1,1,3.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
78533,1,0,15,0,4.00,1.00,0,1,1,1,1,0,1,0,1,1,2.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00
274822,1,0,17,0,3.00,1.00,0,0,0,1,1,0,0,0,1,1,2.00,3.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.

## 1. MODELO INICIAL

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# --- Entrenamiento del modelo ---
rfr = RandomForestRegressor(
    n_estimators=100, 
    max_depth=10,
    random_state=42, 
    n_jobs=-1, 
    oob_score=True
)
rfr.fit(icfesx_original, y_train)

# --- Evaluación con OOB Score ---
# 'oob_score_' es una estimación de la puntuación de validación cruzada
print(f"OOB Score: {rfr.oob_score_:.4f}")

# --- Cálculo de RMSE ---
# 1. Realizar predicciones con los datos de entrenamiento
y_pred_train = rfr.predict(icfesx_original)

# 2. Calcular el Error Cuadrático Medio (MSE)
# La función 'mean_squared_error' ya no admite el parámetro 'squared=False'
mse_train = mean_squared_error(y_train, y_pred_train)

# 3. Calcular el RMSE tomando la raíz cuadrada del MSE
# Para esto, se utiliza la función 'sqrt' de numpy
rmse_train = np.sqrt(mse_train)

print(f"RMSE (entrenamiento): {rmse_train:.4f}")

OOB Score: 0.3685
RMSE (entrenamiento): 39.8115


### 1.1 AJUSTE DE HIPERPARÁMETROS

#### 1.1.1 NÚMERO DE ÁRBOLES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Lista de números de árboles a probar
N_ESTIMADORES = list(range(10, 201, 10))  # De 10 a 200 en pasos de 10
oob_scores = []  # Para almacenar los OOB R² Scores
rmse_scores = []  # Para almacenar los errores RMSE

for n_arboles in N_ESTIMADORES:
    print(f'Entrenando con {n_arboles} árboles...')
    
    # Definir el modelo
    bosque = RandomForestRegressor(n_estimators=n_arboles, 
                                   random_state=42, 
                                   oob_score=True,  # Activar OOB
                                   n_jobs=-1)  # Usar todos los núcleos
    
    # Entrenar el modelo
    bosque.fit(icfesx_original, y_train)

    # Obtener el OOB score (R²)
    oob_r2 = bosque.oob_score_
    oob_scores.append(oob_r2)

    # Obtener las predicciones OOB y calcular RMSE
    oob_preds = bosque.oob_prediction_
    rmse = np.sqrt(mean_squared_error(y_train, oob_preds))
    rmse_scores.append(rmse)

# === Encontrar los valores óptimos ===
mejor_oob = max(oob_scores)
mejor_oob_n = N_ESTIMADORES[np.argmax(oob_scores)]

mejor_rmse = min(rmse_scores)
mejor_rmse_n = N_ESTIMADORES[np.argmin(rmse_scores)]

print("\n===== Resultados Óptimos =====")
print(f"Mejor OOB R²: {mejor_oob:.4f} con {mejor_oob_n} árboles")
print(f"Menor RMSE : {mejor_rmse:.4f} con {mejor_rmse_n} árboles")

# === Graficar ===
plt.figure(figsize=(12, 10))

# --- OOB R² ---
plt.subplot(1, 2, 1)
plt.plot(N_ESTIMADORES, oob_scores, marker='o', color='blue')
plt.xlabel('Número de árboles')
plt.ylabel('OOB R² Score')
plt.title('Desempeño del modelo (OOB R²)')
plt.axvline(mejor_oob_n, color='gray', linestyle='--')
plt.text(mejor_oob_n, mejor_oob, f"  {mejor_oob_n} árboles", color="blue")

# Añadir valores encima de los puntos
for i, score in enumerate(oob_scores):
    plt.text(N_ESTIMADORES[i], score + 0.002, f"{score:.3f}",
             ha='center', va='bottom', fontsize=8, color='blue')

# --- RMSE ---
plt.subplot(1, 2, 2)
plt.plot(N_ESTIMADORES, rmse_scores, marker='s', color='red')
plt.xlabel('Número de árboles')
plt.ylabel('RMSE')
plt.title('Error en predicciones OOB (RMSE)')
plt.axvline(mejor_rmse_n, color='gray', linestyle='--')
plt.text(mejor_rmse_n, mejor_rmse, f"  {mejor_rmse_n} árboles", color="red")

# Añadir valores encima de los puntos
for i, score in enumerate(rmse_scores):
    plt.text(N_ESTIMADORES[i], score + 0.2, f"{score:.2f}",
             ha='center', va='bottom', fontsize=8, color='red')

plt.tight_layout()
plt.show()

#### 1.1.2 MÁXIMO DE PROFUNDIDAD DEL ÁRBOL

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

MAX_DEPTH_VALUES = [5, 10, 20, None]  # None significa sin límite de profundidad
oob_r2_scores_depth = []

for max_depth in MAX_DEPTH_VALUES:
    print(f'Entrenando con max_depth={max_depth}...')
    bosque = RandomForestRegressor(n_estimators=100,  
                                   max_depth=max_depth,
                                   random_state=42,
                                   oob_score=True,
                                   n_jobs=-1)
    
    bosque.fit(icfesx_original, y_train)
    oob_r2_scores_depth.append(bosque.oob_score_)

# === Encontrar el mejor OOB R² ===
mejor_oob = max(oob_r2_scores_depth)
mejor_depth = MAX_DEPTH_VALUES[np.argmax(oob_r2_scores_depth)]

print("\n===== Resultados Óptimos =====")
print(f"Mejor OOB R²: {mejor_oob:.4f} con max_depth={mejor_depth}")

# === Graficar resultados ===
x_labels = [str(v) if v is not None else "None" for v in MAX_DEPTH_VALUES]

plt.figure(figsize=(12, 10))
plt.plot(x_labels, oob_r2_scores_depth, marker='o')
plt.xlabel('Máxima Profundidad')
plt.ylabel('OOB R² Score')
plt.title('Impacto de Máxima Profundidad en el rendimiento')

# Añadir valores encima de cada punto
for i, score in enumerate(oob_r2_scores_depth):
    plt.text(x_labels[i], score + 0.002, f"{score:.3f}", 
             ha='center', va='bottom', fontsize=8, color='blue')

plt.show()

#### 1.1.3 NÚMERO DE VARIABLES

In [ ]:
MAX_FEATURES_VALUES = ["sqrt", "log2"]  # Valores válidos en regresión
oob_r2_scores_features = []

for max_features in MAX_FEATURES_VALUES:
    print(f'Entrenando con max_features={max_features}...')
    bosque = RandomForestRegressor(n_estimators=100,  
                                   max_depth=20,  # Un valor fijo
                                   max_features=max_features,
                                   random_state=42,
                                   oob_score=True,
                                   n_jobs=-1)
    
    bosque.fit(icfesx_original, y_train)
    oob_r2_scores_features.append(bosque.oob_score_)

# Graficar resultados
plt.figure(figsize=(12, 10))
plt.plot(MAX_FEATURES_VALUES, oob_r2_scores_features, marker='o')
plt.xlabel('Max Features')
plt.ylabel('OOB R² Score')
plt.title('Impacto de max_features en el rendimiento')
plt.show()

## 2. MODELO CON HIPERPARÁMETROS ÓPTIMOS

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# --- Entrenamiento del modelo ---
rfr = RandomForestRegressor(
    n_estimators=100, 
    max_depth=20,
    max_features='sqrt',
    random_state=42, 
    n_jobs=-1, 
    oob_score=True
)
rfr.fit(icfesx_original, y_train)

# --- Evaluación con OOB Score ---
# 'oob_score_' es una estimación de la puntuación de validación cruzada
print(f"OOB Score: {rfr.oob_score_:.4f}")

# --- Cálculo de RMSE ---
# 1. Realizar predicciones con los datos de entrenamiento
y_pred_train = rfr.predict(icfesx_original)

# 2. Calcular el Error Cuadrático Medio (MSE)
mse_train = mean_squared_error(y_train, y_pred_train)

# 3. Calcular el RMSE tomando la raíz cuadrada del MSE
# Para esto, se utiliza la función 'sqrt' de numpy
rmse_train = np.sqrt(mse_train)

print(f"RMSE (entrenamiento): {rmse_train:.4f}")

### 2.1 VARIABLES MÁS IMPORTANTES

In [ ]:
# Obtener importancias
importances = rfr.feature_importances_

# Crear un DataFrame ordenado de mayor a menor importancia
feature_importance_df = pd.DataFrame({
    'Feature': icfesx_original.columns,  
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Mostrar las 10 más importantes
print(feature_importance_df.head(10))

In [ ]:
# Graficar las 10 variables más importantes
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df[:10], x='Importance', y='Feature', palette='viridis')
plt.title("Top 10 Variables Más Importantes en Random Forest")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.show()

### 2.2 VARIABLES MENOS IMPORTANTES

In [ ]:
importances = pd.DataFrame({
    'variable': icfesx_original.columns,
    'importancia': rfr.feature_importances_
})
importances.sort_values(by='importancia', inplace=True)
print(importances.head(10))  # las 10 menos importantes

In [ ]:
# Graficar las 10 variables más importantes
plt.figure(figsize=(10, 6))
sns.barplot(data=importances[:10], x='Importance', y='Feature', palette='viridis')
plt.title("Top 10 Variables Más Importantes en Random Forest")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.show()

## 3. EVALUACIÓN EXTRA DEL MODELO

### 3.1 MÉTRICAS EN CONTEXTO (RMSE y OOB SCORE)

In [ ]:
# =============================================================================
# PRIMER ANÁLISIS: MÉTRICAS CONTEXTUALIZADAS
# =============================================================================
print("="*60)
print("1. ANÁLISIS DE MÉTRICAS CONTEXTUALIZADAS")
print("="*60)

# Obtener estadísticas descriptivas del puntaje ICFES
puntaje_stats = y_train.describe()
print("\nEstadísticas descriptivas del puntaje ICFES:")
print(puntaje_stats)

# Calcular desviación estándar
std_puntajes = y_train.std()
print(f"\nDesviación estándar de los puntajes: {std_puntajes:.2f}")

# Tu RMSE actual
rmse_actual = 32.21  # El que obtuviste

# Calcular RMSE como proporción de la desviación estándar
rmse_normalized = rmse_actual / std_puntajes
print(f"RMSE como proporción de la desviación estándar: {rmse_normalized:.3f}")
print(f"Esto significa que tu error típico es de {rmse_normalized:.1f} desviaciones estándar")

# Interpretación contextualizada
rango_real = y_train.max() - y_train.min()
error_porcentual_real = (rmse_actual / rango_real) * 100
print(f"\nRango real de puntajes en tus datos: {rango_real:.1f} puntos")
print(f"RMSE como % del rango real: {error_porcentual_real:.1f}%")

# Crear visualización de la distribución de puntajes
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(y_train, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(y_train.mean(), color='red', linestyle='--', 
           label=f'Media: {icfes["puntaje_global"].mean():.1f}')
plt.axvline(y_train.mean() + rmse_actual, color='orange', linestyle='--',
           label=f'Media + RMSE: {icfes["puntaje_global"].mean() + rmse_actual:.1f}')
plt.axvline(y_train.mean() - rmse_actual, color='orange', linestyle='--',
           label=f'Media - RMSE: {icfes["puntaje_global"].mean() - rmse_actual:.1f}')
plt.xlabel('Puntaje ICFES')
plt.ylabel('Frecuencia')
plt.title('Distribución de Puntajes ICFES\ncon Bandas de Error RMSE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot(y_train)
plt.ylabel('Puntaje ICFES')
plt.title('Boxplot de Puntajes ICFES')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 EXÁMEN DE RESIDUOS

In [ ]:
from scipy import stats
# =============================================================================
# SEGUNDO ANÁLISIS: EXAMEN DE RESIDUOS
# =============================================================================
print("\n" + "="*60)
print("2. ANÁLISIS DE RESIDUOS DEL MODELO")
print("="*60)

# Generar predicciones con el modelo entrenado
y_pred_train = rfr.predict(icfesx_original)

# Calcular residuos reales
residuos = y_train - y_pred_train

# Estadísticas descriptivas de los residuos
print(f"Estadísticas de los residuos:")
print(f"Media de residuos: {np.mean(residuos):.3f} (idealmente cerca de 0)")
print(f"Desviación estándar de residuos: {np.std(residuos):.3f}")
print(f"RMSE calculado de residuos: {np.sqrt(np.mean(residuos**2)):.3f}")


# Crear gráficos de diagnóstico de residuos
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Residuos vs Valores Predichos
axes[0,0].scatter(y_pred_train, residuos, alpha=0.6, color='blue')
axes[0,0].axhline(y=0, color='red', linestyle='--')
axes[0,0].set_xlabel('Valores Predichos')
axes[0,0].set_ylabel('Residuos')
axes[0,0].set_title('Residuos vs Valores Predichos\n(Busca patrones sistemáticos)')
axes[0,0].grid(True, alpha=0.3)

# Añadir línea de tendencia para detectar patrones
z = np.polyfit(y_pred_train, residuos, 1)
p = np.poly1d(z)
axes[0,0].plot(sorted(y_pred_train), p(sorted(y_pred_train)), "r--", alpha=0.8)

# 2. Histograma de residuos
axes[0,1].hist(residuos, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0,1].set_xlabel('Residuos')
axes[0,1].set_ylabel('Frecuencia')
axes[0,1].set_title('Distribución de Residuos\n(Debería ser aproximadamente normal)')
axes[0,1].grid(True, alpha=0.3)

# 3. Q-Q plot para normalidad
stats.probplot(residuos, dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot de Residuos\n(Evalúa normalidad)')
axes[1,0].grid(True, alpha=0.3)

# 4. Residuos vs Valores Reales
axes[1,1].scatter(y_train, residuos, alpha=0.6, color='purple')
axes[1,1].axhline(y=0, color='red', linestyle='--')
axes[1,1].set_xlabel('Valores Reales')
axes[1,1].set_ylabel('Residuos')
axes[1,1].set_title('Residuos vs Valores Reales')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Test de normalidad de residuos
shapiro_stat, shapiro_p = stats.shapiro(residuos[:5000] if len(residuos) > 5000 else residuos)
print(f"\nTest de Shapiro-Wilk para normalidad de residuos:")
print(f"Estadístico: {shapiro_stat:.4f}, p-value: {shapiro_p:.4f}")
if shapiro_p > 0.05:
    print("Los residuos siguen una distribución normal (buena señal)")
else:
    print("Los residuos NO siguen una distribución normal (posible problema)")

### 3.3 EVALUACIÓN DE ERRORES POR RANGOS

In [ ]:
# =============================================================================
# TERCER ANÁLISIS: EVALUACIÓN CONTEXTUAL POR RANGOS DE PUNTAJE
# =============================================================================
print("\n" + "="*60)
print("3. ANÁLISIS CONTEXTUAL POR RANGOS DE PUNTAJE")
print("="*60)

# Copiar dataset de entrenamiento y añadir target real y predicciones
icfes_copy = icfesx_original.copy()
icfes_copy['puntaje_real'] = y_train.values
icfes_copy['puntaje_pred'] = y_pred_train
icfes_copy['residuo'] = icfes_copy['puntaje_real'] - icfes_copy['puntaje_pred']

# Crear categorías de puntaje real
icfes_copy['categoria_puntaje'] = pd.cut(icfes_copy['puntaje_real'], 
                                        bins=5, 
                                        labels=['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto'])

# Análisis por categorías
print("Distribución por categorías de puntaje:")
print(icfes_copy['categoria_puntaje'].value_counts().sort_index())

# Calcular errores reales por categoría
categoria_stats = []
for categoria in ['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto']:
    mask = icfes_copy['categoria_puntaje'] == categoria
    if mask.sum() > 0:
        puntajes_categoria = icfes_copy.loc[mask, 'puntaje_real']
        residuos_categoria = icfes_copy.loc[mask, 'residuo']

        rmse_categoria = np.sqrt(np.mean(residuos_categoria**2))
        
        categoria_stats.append({
            'Categoria': categoria,
            'N_estudiantes': mask.sum(),
            'Puntaje_promedio': puntajes_categoria.mean(),
            'RMSE_categoria': rmse_categoria,
            'Error_relativo': (rmse_categoria / puntajes_categoria.mean()) * 100
        })

df_categoria_stats = pd.DataFrame(categoria_stats)
print(f"\nAnálisis de error por categoría de puntaje:")
print(df_categoria_stats.to_string(index=False, float_format='%.2f'))

# Visualización del análisis contextual
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
df_categoria_stats.plot(x='Categoria', y='RMSE_categoria', kind='bar', ax=plt.gca(), color='coral')
plt.title('RMSE por Categoría de Puntaje')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
df_categoria_stats.plot(x='Categoria', y='Error_relativo', kind='bar', ax=plt.gca(), color='lightblue')
plt.title('Error Relativo por Categoría')
plt.ylabel('Error Relativo (%)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
icfes_copy['categoria_puntaje'].value_counts().sort_index().plot(kind='bar', color='lightgreen')
plt.title('Distribución de Estudiantes por Categoría')
plt.ylabel('Número de Estudiantes')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# =============================================================================
# INTERPRETACIÓN Y RECOMENDACIONES
# =============================================================================
print("\n" + "="*60)
print("INTERPRETACIÓN Y RECOMENDACIONES")
print("="*60)

# Calcular métricas actualizadas para el resumen
rmse_actual = 32.21
oob_score_actual = 0.3872
error_porcentual_real = (rmse_actual / rango_real) * 100
rmse_normalized = rmse_actual / std_puntajes

print(f"""
RESUMEN DE TU MODELO OPTIMIZADO:
- RMSE: {rmse_actual} puntos
- OOB Score (R²): {oob_score_actual:.4f}
- Error como % del rango real: {error_porcentual_real:.1f}%
- Error en desviaciones estándar: {rmse_normalized:.3f}

INTERPRETACIÓN:
1. Tu RMSE optimizado de {rmse_actual} puntos representa {rmse_normalized:.3f} desviaciones estándar,
   lo cual es EXCELENTE para predicción de rendimiento académico.

2. El OOB Score de {oob_score_actual:.1%} significa que tu modelo explica el {oob_score_actual*100:.1f}% de la varianza.
   Para variables socioeconómicas prediciendo ICFES, esto es muy respetable.

3. El {100-oob_score_actual*100:.1f}% restante de la varianza se debe a factores no incluidos en tu modelo:
   - Calidad específica de la educación recibida
   - Esfuerzo y motivación individual
   - Habilidades cognitivas específicas
   - Preparación para el examen
   - Factores del día del examen

MEJORAS LOGRADAS CON LA OPTIMIZACIÓN:
- Reducción del RMSE de 39 a {rmse_actual} puntos ({((39-rmse_actual)/39)*100:.1f}% de mejora)
- Error normalizado mejoró de 0.769 a {rmse_normalized:.3f} desviaciones estándar
- Mejora en capacidad predictiva manteniendo interpretabilidad del modelo

RECOMENDACIÓN FINAL:
Tu modelo optimizado ES ALTAMENTE ÚTIL. Un R² de {oob_score_actual:.3f} en ciencias sociales 
es significativo, y el RMSE de {rmse_actual} puntos es excelente considerando la complejidad 
del fenómeno que estás modelando. Los hiperparámetros elegidos (max_depth=20, max_features='sqrt') 
han logrado un balance óptimo entre capacidad predictiva y control de sobreajuste.
""")

## 4. PREPARACIÓN DE DATOS DE PRUEBA

In [10]:
X_test.shape

(84427, 36)

In [11]:
X_test.sample(5)

,nacionalidad,region,etnia_estudiante,edad,presento_fuera_edad,num_personas_casa,num_cuartos_casa,estrato_casa,tiempo_internet,internet,tv,computador,lavadora,microndas,carro,moto,consola,num_libros,tiempo_lectura,freq_leche_derivados,freq_carne_pescado_similares,freq_cereales_frutos_legumbres,nivel_edu_padre,actividad_padre,nivel_edu_madre,actividad_madre,horas_trabajo_semanal,colegio_genero,colegio_publico,colegio_area,colegio_jornada,nse_estudiante,nse_colegio,educacion_padres,perfil_lector,actividad_padres
80358,Colombia,Pacífica,No,16,0,Hogar grande,3.00,2.00,Promedio,1,1,1,1,1,0,1,0,11-25,No lee,Insuficiente,Aceptable,Insuficiente,Bachiller,Sin Información,Profesional Inc,Microempresario,No Trabaja,Mixto,No Oficial,Urbano,Tradicional,3.00,2.00,Al Menos Un Bachiller,"Poco Apoyo, Poco Habito",Empresarios/Independientes
10474,Colombia,Andina,No,38,1,Hogar tradicional,3.00,2.00,Moderado,0,1,1,1,1,0,0,0,0-10,No lee,Óptimo,Óptimo,Aceptable,Primaria Inc,No Aplica,Primaria Inc,Sector Primario,Tiempo Completo,Mixto,No Oficial,Urbano,Validación,2.00,3.00,Educación Primaria Incompleta,"Poco Apoyo, Poco Habito",No Aplica
67559,Colombia,Caribe,No,16,0,Hogar tradicional,2.00,1.00,Moderado,0,0,0,1,0,0,0,0,11-25,0-30 min,Óptimo,Insuficiente,Muy Insuficiente,Tecnico/Tecnologo,Sin Actividad Remunerada,Bachiller Inc,Trabajador Independiente,No Trabaja,Mixto,Oficial,Urbano,Unica,2.00,2.00,Educación Técnica,"Poco Apoyo, Poco Habito",Empresarios/Independientes
8541,Colombia,Andina,No,18,0,Hogar grande,3.00,3.00,Moderado,1,1,1,1,0,0,0,0,11-25,0-30 min,Óptimo,Óptimo,Aceptable,Primaria,Trabajadores Operativos,Primaria,No Aplica,Trabajo Ocasional,Mixto,No Oficial,Urbano,Tradicional,2.00,2.00,Educación Primaria,"Poco Apoyo, Poco Habito",No Aplica
36736,Colombia,Andina,No,18,0,Hogar tradicional,2.00,3.00,Mucho,1,1,1,1,1,0,0,0,26-100,0-30 min,Óptimo,Aceptable,Insuficiente,Bachiller,Trabajadores Operativos,Tecnico/Tecnologo,Trabajador Independiente,No Trabaja,Mixto,Oficial,Urbano,Unica,3.00,3.00,Educación Técnica,"Buen Apoyo, Poco Habito",Empresarios/Independientes


In [12]:
X_test = X_test.drop(columns=['educacion_padres', 'perfil_lector', 'actividad_padres'])
X_test.sample(5)

,nacionalidad,region,etnia_estudiante,edad,presento_fuera_edad,num_personas_casa,num_cuartos_casa,estrato_casa,tiempo_internet,internet,tv,computador,lavadora,microndas,carro,moto,consola,num_libros,tiempo_lectura,freq_leche_derivados,freq_carne_pescado_similares,freq_cereales_frutos_legumbres,nivel_edu_padre,actividad_padre,nivel_edu_madre,actividad_madre,horas_trabajo_semanal,colegio_genero,colegio_publico,colegio_area,colegio_jornada,nse_estudiante,nse_colegio
25991,Colombia,Andina,No,18,0,Hogar tradicional,3.00,1.00,Promedio,0,0,0,1,0,0,0,0,0-10,1-2h,Aceptable,Óptimo,Insuficiente,Primaria Inc,Microempresario,Ninguna,Trabajadores Operativos,Trabajo Ocasional,Mixto,Oficial,Urbano,Validación,1.00,2.00
64484,Colombia,Orinoquía,No,21,1,Hogar tradicional,4.00,2.00,Poco,1,1,0,0,0,1,1,0,26-100,30-60 min,Óptimo,Óptimo,Insuficiente,Bachiller,Trabajadores Operativos,Tecnico/Tecnologo Inc,Sector Primario,Tiempo Parcial Reducido,Mixto,No Oficial,Urbano,Validación,3.00,2.00
40502,Colombia,Caribe,No,17,0,Hogar grande,2.00,2.00,Promedio,1,1,1,1,0,0,0,0,26-100,0-30 min,Óptimo,Óptimo,Óptimo,Profesional Inc,Sin Actividad Remunerada,Bachiller,Trabajadores Operativos,No Trabaja,Mixto,Oficial,Urbano,Tradicional,3.00,2.00
20798,Colombia,Andina,No,17,0,Hogar grande,4.00,2.00,Promedio,1,1,1,0,0,0,0,0,0-10,30-60 min,Aceptable,Aceptable,Muy Insuficiente,Primaria,Trabajadores Operativos,Profesional,No Aplica,No Trabaja,Mixto,Oficial,Rural,Tradicional,2.00,2.00
56310,Colombia,Caribe,No,16,0,Hogar grande,2.00,2.00,Promedio,1,0,1,1,1,0,1,0,11-25,30-60 min,Óptimo,Óptimo,Aceptable,Tecnico/Tecnologo Inc,Microempresario,Tecnico/Tecnologo Inc,Trabajadores Operativos,No Trabaja,Mixto,Oficial,Urbano,Tradicional,3.00,3.00


### 4.1 CODIFICACIÓN DE VARIABLES

#### 4.1.1 ENCODING BINARIOS

In [13]:
def codificacion_dummy(df): # Renombrar columnas 
    df.rename(columns={'nacionalidad': 'colombiano', 'etnia_estudiante': 'etnia'}, inplace=True) 
    # Codificación binaria 
    df['colombiano'] = (df['colombiano'] == 'Colombia').astype(int) 
    df['etnia'] = (df['etnia'] != 'No').astype(int) 
    return df

codificacion_dummy(X_test)

X_test.sample(5)

,colombiano,region,etnia,edad,presento_fuera_edad,num_personas_casa,num_cuartos_casa,estrato_casa,tiempo_internet,internet,tv,computador,lavadora,microndas,carro,moto,consola,num_libros,tiempo_lectura,freq_leche_derivados,freq_carne_pescado_similares,freq_cereales_frutos_legumbres,nivel_edu_padre,actividad_padre,nivel_edu_madre,actividad_madre,horas_trabajo_semanal,colegio_genero,colegio_publico,colegio_area,colegio_jornada,nse_estudiante,nse_colegio
21217,1,Caribe,0,17,0,Hogar grande,2.00,1.00,Promedio,0,1,0,1,0,0,0,0,26-100,30-60 min,Insuficiente,Aceptable,Insuficiente,Profesional Inc,Sin Actividad Remunerada,Profesional Inc,Trabajadores Operativos,No Trabaja,Mixto,Oficial,Urbano,Tradicional,2.00,3.00
21257,1,Caribe,0,17,0,Hogar grande,1.00,2.00,Mucho,1,1,0,1,0,0,0,1,0-10,0-30 min,Insuficiente,Insuficiente,Insuficiente,Bachiller Inc,Trabajadores Operativos,Bachiller Inc,No Aplica,No Trabaja,Mixto,Oficial,Urbano,Tradicional,2.00,2.00
22524,1,Caribe,0,18,0,Más de 1 hogar,3.00,1.00,Mucho,1,1,0,1,0,0,0,0,0-10,30-60 min,Insuficiente,Óptimo,Muy Insuficiente,Ninguna,Trabajadores Operativos,Bachiller,Sector Primario,No Trabaja,Mixto,Oficial,Rural,Unica,2.00,2.00
61111,1,Andina,0,16,0,Hogar tradicional,2.00,2.00,Promedio,0,0,0,1,1,0,1,0,11-25,1-2h,Óptimo,Óptimo,Insuficiente,Tecnico/Tecnologo,Sin Actividad Remunerada,Profesional,Profesionales,No Trabaja,Mixto,Oficial,Rural,Tradicional,3.00,2.00
53983,1,Andina,0,19,0,Hogar grande,2.00,1.00,Moderado,0,1,0,1,0,1,0,0,11-25,30-60 min,Aceptable,Aceptable,Insuficiente,Primaria Inc,Sin Actividad Remunerada,Primaria Inc,Trabajadores Operativos,Trabajo Ocasional,Mixto,Oficial,Rural,Validación,2.00,2.00


In [14]:
def binarizar_colegio_area_publico(df):
    
    df['colegio_publico'] = np.where(df['colegio_publico'] == 'Oficial', 1, 0)
    df['colegio_area'] = np.where(df['colegio_area'] == 'Urbano', 1, 0)
    
    return df

binarizar_colegio_area_publico(X_test)
X_test[['colegio_publico', 'colegio_area']].head()

X_test.sample(5)

,colombiano,region,etnia,edad,presento_fuera_edad,num_personas_casa,num_cuartos_casa,estrato_casa,tiempo_internet,internet,tv,computador,lavadora,microndas,carro,moto,consola,num_libros,tiempo_lectura,freq_leche_derivados,freq_carne_pescado_similares,freq_cereales_frutos_legumbres,nivel_edu_padre,actividad_padre,nivel_edu_madre,actividad_madre,horas_trabajo_semanal,colegio_genero,colegio_publico,colegio_area,colegio_jornada,nse_estudiante,nse_colegio
75208,1,Caribe,0,17,0,Hogar tradicional,3.00,4.00,Mucho,1,1,1,1,1,0,0,0,11-25,0-30 min,Óptimo,Óptimo,Aceptable,Postgrado,Profesionales,Profesional,Profesionales,No Trabaja,Mixto,0,1,Completa,4.00,4.00
52499,1,Andina,0,17,0,Más de 1 hogar,3.00,1.00,Poco,0,0,0,1,1,0,0,0,0-10,30-60 min,Insuficiente,Insuficiente,Insuficiente,Bachiller Inc,Microempresario,Bachiller Inc,Trabajador Independiente,No Trabaja,Mixto,1,1,Completa,2.00,2.00
35404,1,Caribe,0,20,1,Más de 1 hogar,3.00,2.00,Poco,0,0,0,0,0,0,0,0,0-10,30-60 min,Insuficiente,Aceptable,Óptimo,Ninguna,Trabajadores Operativos,Bachiller,Sin Actividad Remunerada,Medio Tiempo,Mixto,1,1,Tradicional,1.00,2.00
36988,1,Andina,0,19,0,Hogar tradicional,3.00,2.00,Mucho,1,1,1,0,0,0,0,0,0-10,1-2h,Óptimo,Óptimo,Aceptable,Primaria Inc,Sin Actividad Remunerada,Bachiller Inc,Trabajadores Operativos,Tiempo Parcial Reducido,Mixto,1,0,Completa,2.00,2.00
20582,1,Andina,0,17,0,Hogar grande,3.00,3.00,Mucho,1,1,1,1,1,1,0,1,26-100,1-2h,Aceptable,Óptimo,Óptimo,Profesional,Sin Actividad Remunerada,Tecnico/Tecnologo,Profesionales,No Trabaja,Mixto,0,1,Completa,4.00,3.00


#### 4.1.2 ENCODING ONE HOT

In [15]:
from sklearn.preprocessing import OneHotEncoder

def aplicar_onehot(df, columnas, sparseout=False, drop=None):

    if isinstance(columnas, str):
        columnas = [columnas]  # convertir en lista si es solo una

    for col in columnas:
        if col not in df.columns:
            raise ValueError(f"La columna '{col}' no existe en el DataFrame.")

    encoder = OneHotEncoder(sparse_output=sparseout, drop=drop)

    # Ajustar y transformar
    encoded = encoder.fit_transform(df[columnas])

    # Crear DataFrame con las columnas codificadas
    feature_names = encoder.get_feature_names_out(columnas)
    encoded_df = pd.DataFrame(encoded, columns=feature_names, index=df.index)

    # Eliminar las columnas originales
    df.drop(columns=columnas, inplace=True)

    # Agregar las nuevas columnas al DataFrame original
    for col in encoded_df.columns:
        df[col] = encoded_df[col]

    return encoder


aplicar_onehot(X_test, ['region', 'tiempo_internet', 'num_personas_casa', 'num_libros', 'tiempo_lectura', 'nivel_edu_padre', 'nivel_edu_madre', 
                        'actividad_padre', 'actividad_madre','horas_trabajo_semanal', 'colegio_genero', 'colegio_jornada', 'freq_leche_derivados', 
                        'freq_carne_pescado_similares', 'freq_cereales_frutos_legumbres'], sparseout=False, drop=None)
X_test.sample(5)

,colombiano,etnia,edad,presento_fuera_edad,num_cuartos_casa,estrato_casa,internet,tv,computador,lavadora,microndas,carro,moto,consola,colegio_publico,colegio_area,nse_estudiante,nse_colegio,region_Amazónica,region_Andina,region_Caribe,region_Orinoquía,region_Pacífica,tiempo_internet_Moderado,tiempo_internet_Mucho,tiempo_internet_Ninguno,tiempo_internet_Poco,tiempo_internet_Promedio,num_personas_casa_Hogar grande,num_personas_casa_Hogar tradicional,num_personas_casa_Más de 1 hogar,num_libros_0-10,num_libros_11-25,num_libros_26-100,num_libros_>100,tiempo_lectura_0-30 min,tiempo_lectura_1-2h,tiempo_lectura_30-60 min,tiempo_lectura_>2h,tiempo_lectura_No lee,nivel_edu_padre_Bachiller,nivel_edu_padre_Bachiller Inc,nivel_edu_padre_Ninguna,nivel_edu_padre_No aplica,nivel_edu_padre_Postgrado,nivel_edu_padre_Primaria,nivel_edu_padre_Primaria Inc,nivel_edu_padre_Profesional,nivel_edu_padre_Profesional Inc,nivel_edu_padre_Tecnico/Tecnologo,nivel_edu_padre_Tecnico/Tecnologo Inc,nivel_edu_madre_Bachiller,nivel_edu_madre_Bachiller Inc,nivel_edu_madre_Ninguna,nivel_edu_madre_No aplica,nivel_edu_madre_Postgrado,nivel_edu_madre_Primaria,nivel_edu_madre_Primaria Inc,nivel_edu_madre_Profesional,nivel_edu_madre_Profesional Inc,nivel_edu_madre_Tecnico/Tecnologo,nivel_edu_madre_Tecnico/Tecnologo Inc,actividad_padre_Directivos,actividad_padre_Microempresario,actividad_padre_No Aplica,actividad_padre_Pensionado,actividad_padre_Profesionales,actividad_padre_Sector Primario,actividad_padre_Sin Actividad Remunerada,actividad_padre_Sin Información,actividad_padre_Trabajador Independiente,actividad_padre_Trabajadores Operativos,actividad_madre_Directivos,actividad_madre_Microempresario,actividad_madre_No Aplica,actividad_madre_Pensionado,actividad_madre_Profesionales,actividad_madre_Sector Primario,actividad_madre_Sin Actividad Remunerada,actividad_madre_Sin Información,actividad_madre_Trabajador Independiente,actividad_madre_Trabajadores Operativos,horas_trabajo_semanal_Medio Tiempo,horas_trabajo_semanal_No Trabaja,horas_trabajo_semanal_Tiempo Completo,horas_trabajo_semanal_Tiempo Parcial Reducido,horas_trabajo_semanal_Trabajo Ocasional,colegio_genero_Femenino,colegio_genero_Masculino,colegio_genero_Mixto,colegio_jornada_Completa,colegio_jornada_Tradicional,colegio_jornada_Unica,colegio_jornada_Validación,freq_leche_derivados_Aceptable,freq_leche_derivados_Insuficiente,freq_leche_derivados_Muy Insuficiente,freq_leche_derivados_Óptimo,freq_carne_pescado_similares_Aceptable,freq_carne_pescado_similares_Insuficiente,freq_carne_pescado_similares_Muy Insuficiente,freq_carne_pescado_similares_Óptimo,freq_cereales_frutos_legumbres_Aceptable,freq_cereales_frutos_legumbres_Insuficiente,freq_cereales_frutos_legumbres_Muy Insuficiente,freq_cereales_frutos_legumbres_Óptimo
18171,1,0,17,0,3.00,2.00,1,1,1,0,1,0,0,0,1,1,2.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00
77605,1,0,18,0,2.00,2.00,0,1,1,1,0,1,0,0,1,0,3.00,3.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
53902,1,0,17,0,2.00,2.00,1,1,0,0,0,1,0,0,0,1,2.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.

#### 4.1.3 RENOMBRAR VARIABLES

In [16]:
rename_dict = {
    # Regiones
    "region_Amazónica": "region_amazonica",
    "region_Andina": "region_andina",
    "region_Caribe": "region_caribe",
    "region_Orinoquía": "region_orinoquia",
    "region_Pacífica": "region_pacifica",

    # Tiempo en internet
    "tiempo_internet_Moderado": "internet_moderado",
    "tiempo_internet_Mucho": "internet_mucho",
    "tiempo_internet_Ninguno": "internet_ninguno",
    "tiempo_internet_Poco": "internet_poco",
    "tiempo_internet_Promedio": "internet_promedio",

    # Personas en casa
    "num_personas_casa_Hogar grande": "hogar_grande",
    "num_personas_casa_Hogar tradicional": "hogar_tradicional",
    "num_personas_casa_Más de 1 hogar": "hogar_multiple",

    # Libros
    "num_libros_0-10": "libros_0_10",
    "num_libros_11-25": "libros_11_25",
    "num_libros_26-100": "libros_26_100",
    "num_libros_>100": "libros_mas_100",

    # Tiempo lectura
    "tiempo_lectura_0-30 min": "lectura_0_30min",
    "tiempo_lectura_30-60 min": "lectura_30_60min",
    "tiempo_lectura_1-2h": "lectura_1_2h",
    "tiempo_lectura_>2h": "lectura_mas_2h",
    "tiempo_lectura_No lee": "lectura_nula",

    # Nivel educativo padre
    "nivel_edu_padre_Bachiller": "edu_padre_bachiller",
    "nivel_edu_padre_Bachiller Inc": "edu_padre_bachiller_inc",
    "nivel_edu_padre_Ninguna": "edu_padre_ninguna",
    "nivel_edu_padre_No aplica": "edu_padre_no_aplica",
    "nivel_edu_padre_Postgrado": "edu_padre_postgrado",
    "nivel_edu_padre_Primaria": "edu_padre_primaria",
    "nivel_edu_padre_Primaria Inc": "edu_padre_primaria_inc",
    "nivel_edu_padre_Profesional": "edu_padre_profesional",
    "nivel_edu_padre_Profesional Inc": "edu_padre_profesional_inc",
    "nivel_edu_padre_Tecnico/Tecnologo": "edu_padre_tecnico",
    "nivel_edu_padre_Tecnico/Tecnologo Inc": "edu_padre_tecnico_inc",

    # Nivel educativo madre
    "nivel_edu_madre_Bachiller": "edu_madre_bachiller",
    "nivel_edu_madre_Bachiller Inc": "edu_madre_bachiller_inc",
    "nivel_edu_madre_Ninguna": "edu_madre_ninguna",
    "nivel_edu_madre_No aplica": "edu_madre_no_aplica",
    "nivel_edu_madre_Postgrado": "edu_madre_postgrado",
    "nivel_edu_madre_Primaria": "edu_madre_primaria",
    "nivel_edu_madre_Primaria Inc": "edu_madre_primaria_inc",
    "nivel_edu_madre_Profesional": "edu_madre_profesional",
    "nivel_edu_madre_Profesional Inc": "edu_madre_profesional_inc",
    "nivel_edu_madre_Tecnico/Tecnologo": "edu_madre_tecnico",
    "nivel_edu_madre_Tecnico/Tecnologo Inc": "edu_madre_tecnico_inc",

    # Actividad padre
    "actividad_padre_Directivos": "act_padre_directivos",
    "actividad_padre_Microempresario": "act_padre_microempresario",
    "actividad_padre_No Aplica": "act_padre_no_aplica",
    "actividad_padre_Pensionado": "act_padre_pensionado",
    "actividad_padre_Profesionales": "act_padre_profesionales",
    "actividad_padre_Sector Primario": "act_padre_sector_primario",
    "actividad_padre_Sin Actividad Remunerada": "act_padre_sin_trabajo",
    "actividad_padre_Sin Información": "act_padre_sin_info",
    "actividad_padre_Trabajador Independiente": "act_padre_independiente",
    "actividad_padre_Trabajadores Operativos": "act_padre_operativos",

    # Actividad madre
    "actividad_madre_Directivos": "act_madre_directivos",
    "actividad_madre_Microempresario": "act_madre_microempresario",
    "actividad_madre_No Aplica": "act_madre_no_aplica",
    "actividad_madre_Pensionado": "act_madre_pensionado",
    "actividad_madre_Profesionales": "act_madre_profesionales",
    "actividad_madre_Sector Primario": "act_madre_sector_primario",
    "actividad_madre_Sin Actividad Remunerada": "act_madre_sin_trabajo",
    "actividad_madre_Sin Información": "act_madre_sin_info",
    "actividad_madre_Trabajador Independiente": "act_madre_independiente",
    "actividad_madre_Trabajadores Operativos": "act_madre_operativos",

    # Horas trabajo semanal
    "horas_trabajo_semanal_Medio Tiempo": "trabajo_medio_tiempo",
    "horas_trabajo_semanal_No Trabaja": "trabajo_no",
    "horas_trabajo_semanal_Tiempo Completo": "trabajo_tiempo_completo",
    "horas_trabajo_semanal_Tiempo Parcial Reducido": "trabajo_parcial_reducido",
    "horas_trabajo_semanal_Trabajo Ocasional": "trabajo_ocacional",

    # Colegio genero y jornada
    "colegio_genero_Femenino": "colegio_femenino",
    "colegio_genero_Masculino": "colegio_masculino",
    "colegio_genero_Mixto": "colegio_mixto",
    "colegio_jornada_Completa": "jornada_completa",
    "colegio_jornada_Tradicional": "jornada_tradicional",
    "colegio_jornada_Unica": "jornada_unica",
    "colegio_jornada_Validación": "jornada_validacion",

    # Frecuencia alimentos
    "freq_leche_derivados_Aceptable": "leche_aceptable",
    "freq_leche_derivados_Insuficiente": "leche_insuficiente",
    "freq_leche_derivados_Muy Insuficiente": "leche_muy_insuficiente",
    "freq_leche_derivados_Óptimo": "leche_optimo",

    "freq_carne_pescado_similares_Aceptable": "carne_aceptable",
    "freq_carne_pescado_similares_Insuficiente": "carne_insuficiente",
    "freq_carne_pescado_similares_Muy Insuficiente": "carne_muy_insuficiente",
    "freq_carne_pescado_similares_Óptimo": "carne_optimo",

    "freq_cereales_frutos_legumbres_Aceptable": "cereales_aceptable",
    "freq_cereales_frutos_legumbres_Insuficiente": "cereales_insuficiente",
    "freq_cereales_frutos_legumbres_Muy Insuficiente": "cereales_muy_insuficiente",
    "freq_cereales_frutos_legumbres_Óptimo": "cereales_optimo"
}

X_test.rename(columns=rename_dict, inplace=True)
X_test.sample(5)

,colombiano,etnia,edad,presento_fuera_edad,num_cuartos_casa,estrato_casa,internet,tv,computador,lavadora,microndas,carro,moto,consola,colegio_publico,colegio_area,nse_estudiante,nse_colegio,region_amazonica,region_andina,region_caribe,region_orinoquia,region_pacifica,internet_moderado,internet_mucho,internet_ninguno,internet_poco,internet_promedio,hogar_grande,hogar_tradicional,hogar_multiple,libros_0_10,libros_11_25,libros_26_100,libros_mas_100,lectura_0_30min,lectura_1_2h,lectura_30_60min,lectura_mas_2h,lectura_nula,edu_padre_bachiller,edu_padre_bachiller_inc,edu_padre_ninguna,edu_padre_no_aplica,edu_padre_postgrado,edu_padre_primaria,edu_padre_primaria_inc,edu_padre_profesional,edu_padre_profesional_inc,edu_padre_tecnico,edu_padre_tecnico_inc,edu_madre_bachiller,edu_madre_bachiller_inc,edu_madre_ninguna,edu_madre_no_aplica,edu_madre_postgrado,edu_madre_primaria,edu_madre_primaria_inc,edu_madre_profesional,edu_madre_profesional_inc,edu_madre_tecnico,edu_madre_tecnico_inc,act_padre_directivos,act_padre_microempresario,act_padre_no_aplica,act_padre_pensionado,act_padre_profesionales,act_padre_sector_primario,act_padre_sin_trabajo,act_padre_sin_info,act_padre_independiente,act_padre_operativos,act_madre_directivos,act_madre_microempresario,act_madre_no_aplica,act_madre_pensionado,act_madre_profesionales,act_madre_sector_primario,act_madre_sin_trabajo,act_madre_sin_info,act_madre_independiente,act_madre_operativos,trabajo_medio_tiempo,trabajo_no,trabajo_tiempo_completo,trabajo_parcial_reducido,trabajo_ocacional,colegio_femenino,colegio_masculino,colegio_mixto,jornada_completa,jornada_tradicional,jornada_unica,jornada_validacion,leche_aceptable,leche_insuficiente,leche_muy_insuficiente,leche_optimo,carne_aceptable,carne_insuficiente,carne_muy_insuficiente,carne_optimo,cereales_aceptable,cereales_insuficiente,cereales_muy_insuficiente,cereales_optimo
78400,1,0,17,0,3.00,1.00,0,0,1,1,0,0,0,0,1,0,1.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00
6869,1,0,53,1,3.00,2.00,1,1,1,1,1,0,0,0,0,1,3.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00
74992,1,0,18,0,5.00,2.00,1,1,0,1,1,0,0,1,0,1,2.00,3.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
4191,1,0,20,1,1.00,1.00,0,0,0,1,0,0,0,0,1,0,1.00,2.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
29647,1,0,17,0,3.00,1.00,1,1,1,1,0,0,1,0,1,1,3.00,2.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.0

#### 4.1.4 SOBRESCRIBIR EL ARCHIVO X_test

In [18]:
X_test.to_parquet("C:/Users/CACTU/OneDrive/Escritorio/Proyectos de Análisis de Datos/Python/Icfes 2019/data/procesada/X_test.parquet", index=False, engine="pyarrow")

## 5. MODELO FINAL

In [ ]:
# --- Evaluación con el conjunto de prueba ---
# 1. Realizar predicciones con X_test
y_pred_test = rfr.predict(X_test)

# 2. Calcular el Error Cuadrático Medio (MSE) en test
mse_test = mean_squared_error(y_test, y_pred_test)

# 3. Calcular el RMSE en test
rmse_test = np.sqrt(mse_test)

print(f"RMSE (prueba): {rmse_test:.4f}")

# También puedes calcular el R² en test
r2_test = rfr.score(X_test, y_test)
print(f"R² (prueba): {r2_test:.4f}")

## 6. GUARDAR MODELO

In [ ]:
import joblib

# Guardar
joblib.dump(rfr, "modelo_icfes.pkl")

NameError: name 'rfr' is not defined

In [4]:
import joblib

# Cargar (cuando quieras usarlo)
modelo = joblib.load("C:/Users/CACTU/OneDrive/Escritorio/Proyectos de Análisis de Datos/Python/Icfes 2019/modelo_icfes.pkl")

print(modelo.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 20, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': True, 'random_state': 42, 'verbose': 0, 'warm_start': False}
